<a href="https://colab.research.google.com/github/BirukovAlex/neto_Python/blob/main/%D0%94%D0%97_Dogs_vs_Cats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import tensorflow as tf
import numpy as np
import os
from glob import glob
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from sklearn.model_selection import train_test_split
import time

In [ ]:
train_files = glob('/content/drive/MyDrive/data/train/*.jpg')
test_files = glob('/content/drive/MyDrive/data/test/*.jpg')

print(f" Найдено тренировочных изображений: {len(train_files)}")
print(f" Найдено тестовых изображений: {len(test_files)}")

 Найдено тренировочных изображений: 25001
 Найдено тестовых изображений: 12500


In [ ]:
train_files, val_files = train_test_split(
    train_files,
    test_size=0.2,
    random_state=42,
    shuffle=True
)

print(f" Обучающая выборка: {len(train_files)} изображений")
print(f" Валидационная выборка: {len(val_files)} изображений")


 Обучающая выборка: 20000 изображений
 Валидационная выборка: 5001 изображений


In [ ]:
print("ДООБУЧЕНИЕ НА БОЛЬШЕЙ ВЫБОРКЕ БЕЗ ПЕРЕГРУЗКИ ПАМЯТИ")

# Увеличиваем выборку но оставляем маленький batch size
print("Создание расширенной выборки...")

def create_larger_dataset(files, batch_size=16):  # Уменьшаем batch_size для памяти
    def load_image(path):
        import cv2
        img = cv2.imread(path)
        if img is None:
            return None
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, IMG_SIZE)
        img = preprocess_input(img.astype(np.float32))
        return img

    def generator():
        for path in files:
            img = load_image(path)
            if img is not None:
                label = 1.0 if 'dog' in os.path.basename(path).lower() else 0.0
                yield img, label

    dataset = tf.data.Dataset.from_generator(
        generator,
        output_signature=(
            tf.TensorSpec(shape=(*IMG_SIZE, 3), dtype=tf.float32),
            tf.TensorSpec(shape=(), dtype=tf.float32)
        )
    )

    dataset = dataset.shuffle(min(1000, len(files)))
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)

    return dataset

# Берем больше данных но с меньшим batch_size
print("Используем 4000 train и 1000 val изображений...")
larger_train_dataset = create_larger_dataset(train_files[:4000])
larger_val_dataset = create_larger_dataset(val_files[:1000])

print("Быстрое дообучение на расширенной выборке...")

# Быстрое обучение на 5 эпохах
history_final = model.fit(
    larger_train_dataset,
    epochs=5,
    validation_data=larger_val_dataset,
    verbose=1
)

# Финальная оценка
final_val_loss, final_val_accuracy = model.evaluate(larger_val_dataset, verbose=0)
print(f"\nФИНАЛЬНЫЕ РЕЗУЛЬТАТЫ НА 4000/1000:")
print(f"Val Loss: {final_val_loss:.4f}")
print(f"Val Accuracy: {final_val_accuracy:.4f}")

# Предсказания на всех тестовых данных
print("\nФинальные предсказания на тестовом наборе...")

def create_test_dataset_generator(files, batch_size=16):
    def load_test_image(path):
        import cv2
        img = cv2.imread(path)
        if img is None:
            return None
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, IMG_SIZE)
        img = preprocess_input(img.astype(np.float32))
        return img

    def test_generator():
        for path in files:
            img = load_test_image(path)
            if img is not None:
                yield img

    dataset = tf.data.Dataset.from_generator(
        test_generator,
        output_signature=tf.TensorSpec(shape=(*IMG_SIZE, 3), dtype=tf.float32)
    )

    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)

    return dataset

# Предсказания на всех тестовых файлах
test_dataset_final = create_test_dataset_generator(test_files)
test_predictions_final = model.predict(test_dataset_final, verbose=1)

print(f"Сгенерировано предсказаний: {len(test_predictions_final)}")

# Создаем финальный файл
print("Создание финального submission.csv...")
with open('submission_final.csv', 'w') as f:
    f.write('id,label\n')
    for i, path in enumerate(test_files):
        if i < len(test_predictions_final):
            file_id = re.search(r'(\d+)\.jpg$', path).group(1)
            f.write(f'{file_id},{test_predictions_final[i][0]:.6f}\n')

print("✅ Файл submission_final.csv создан!")
print(f"✅ Обучение на {4000} тренировочных изображениях")
print(f"✅ Валидация на {1000} изображениях")
print(f"✅ Тестирование на {len(test_files)} изображениях")
print(f"✅ Финальный Log Loss: {final_val_loss:.4f}")

print("\nТеперь submission должен быть принят Kaggle!")import tensorflow as tf
import numpy as np
import os
from glob import glob
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from sklearn.model_selection import train_test_split
import time